In [1]:
import os
from pathlib import Path

NOTEBOOK_PATH: Path = Path(os.path.abspath(''))
PATH_FOR_RESULT: Path = NOTEBOOK_PATH.parent.joinpath('RESULT')

# 获取上海期货交易所（上期所）的历史数据

在[上期所官网](http://www.shfe.com.cn/)点击顶部菜单（如下图中红框所示）：

![上期所历史数据获取1](../pictures/shfe_data_1.png)

然后在出现的页面中，更换下拉框中的年份并点击按钮（如下图中红框所示）。

![上期所历史数据获取2](../pictures/shfe_data_2.png)

多试几次，可以发现历史数据的url是：

`
  http://www.shfe.com.cn/historyData/MarketData_Year_XXXX.zip
`

其中 XXXX 是四位数的年份数字。

于是我们可以用以下代码来获得上期所历史数据：

In [2]:
from pathlib import Path
import datetime as dt

import requests


def download_shfe_history_data(save_path: Path, year: int) -> None:
    """
    下载上海期货交易所（上期所，SHFE）的历史数据。
    :param save_path: 保存的位置。
    :param year: 需要下载数据的年份。
    :return: None.
    """

    # 上期所历史数据 url.
    url: str = 'http://www.shfe.com.cn/historyData/MarketData_Year_{year:4d}.zip'

    # 上期所历史数据从 2009 年开始提供。
    start_year: int = 2009
    
    # 如果参数 <year> 小于 2009 或者大于当前年份，引发异常。
    if year < start_year:
        raise ValueError(f'上期所历史数据自{start_year:4d}年起提供。')
    if year > dt.date.today().year:
        raise ValueError(f'{year}年是未来日期。')

    # 如果参数 <save_path> 不存在，引发异常。
    if not save_path.exists():
        raise FileNotFoundError(f'目录 {save_path} 不存在。')
    
    # 下载。
    response = requests.get(url.format(year=year))
    
    # 如果下载不顺利，引发异常。
    if response.status_code != 200:
        raise requests.exceptions.HTTPError(f'下载 <{url.format(year=year)}> 时发生错误。')
    
    # 保存文件。
    with open(save_path.joinpath(f'SHFE_{year:4d}.zip'), 'wb') as f:
        f.write(response.content)

# 测试。
download_shfe_history_data(save_path=PATH_FOR_RESULT, year=2020)

你应该可以在代码仓库的 RESULT/ 文件夹下面看到名为 <SHFE_2020.zip> 文件。

我们可以自动化下载从2009年（上期所开始提供数据的年份）到现在的所有历史数据：

In [3]:
def download_all_shfe_history_data(save_path: Path) -> None:
    """
    下载上海期货交易所（上期所，SHFE）的全部历史数据。
    :param save_path: 保存的位置。
    :return: None.
    """
    start_year: int = 2009
    this_year: int = dt.date.today().year
    for year in range(start_year, this_year + 1):
        download_shfe_history_data(save_path=save_path, year=year)

# 测试。
download_all_shfe_history_data(save_path=PATH_FOR_RESULT)

很迅速，而且有条理，比手工下载强多了。